In [76]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils

In [77]:
DATA_DIR = "../data/fb15k/"
DUMP_FILE = "../Interpretable-KBC/dumps/fb15k_distmult_dump_norm.pkl"
MODEL_TYPE = "distmult"

In [78]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [109]:
with open(DUMP_FILE,"rb") as f:
    dump = pickle.load(f)

In [80]:
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [110]:
model=models.TypedDM(DUMP_FILE)

180
199


In [82]:
mapped_train_data = utils.map_data(train_data,dump)
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

# Length 1 Rules

In [87]:
rule_similarity_thresh=0.5
relation_count=len(dump['relation_to_id'])

In [31]:
rules_1=[]
for r1 in range(relation_count):
    for r2 in range(relation_count):
        if(r1==r2):
            continue
        get_simi=model.get_relation_similarity(r1,r2)
        if(get_simi>rule_similarity_thresh):
            rules_1.append([r1,r2,get_simi])

# Length 2 Rules

In [127]:
index_head=utils.get_head_index(mapped_train_data)
count_r1_r2,set_r1_r2=utils.get_r1r2_count(mapped_train_data,index_head,get_set=True)
entity_to_rel=utils.get_ent_to_rel(mapped_train_data)

In [128]:
nbrs = NearestNeighbors(n_neighbors=100,metric=model.similarity_relembedding).fit(model.relation_matrix)

In [135]:
count=0
rules_dict={}
for r1 in range(relation_count):
    if(r1%100==0):
        print(r1)
    for r2 in range(relation_count):
        if(r1==r2):
            continue
        pair=(r1,r2)
        if(pair not in count_r1_r2 or count_r1_r2[pair]<10):
            continue
        
        combined_rel=model.dot_relation(r1,r2).reshape((1,-1))
        distances, indices = nbrs.kneighbors(combined_rel)
        count+=1
        bool_arr=[False for i in range(relation_count)]
        for ind in indices[0]:
            bool_arr[ind]=True
        
        cur_dict={}
        entpair_lis=list(set_r1_r2[pair])
        for pair_e1e2 in entpair_lis:
            if pair_e1e2 not in entity_to_rel:
                continue
            for r in entity_to_rel[pair_e1e2]:
                if(bool_arr[r]==False):
                    continue
                if r not in cur_dict:
                    cur_dict[r]=0
                cur_dict[r]+=1
        rules_dict[pair]=cur_dict
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [139]:
print(len(rules_1))
print(len(rules_dict))
print(relation_count)
print(count)

3818
28417
1345
28417


# Saving rules

In [121]:
path1='rules_1.pkl'
with open(path1,"wb") as writefile:
    pickle.dump(rules_1,writefile)


In [138]:
path2='rules_2.pkl'
with open(path2,"wb") as writefile:
    pickle.dump(rules_dict,writefile)

# Prune length 2 rules

In [140]:
#for length 2
def prune_rules(rules_dict,thresh=0.1):
    new_rules=[]
    for pair in rules_dict:
        denom=len(set_r1_r2[pair])
        for r in rules_dict[pair]:
            confidence=rules_dict[pair][r]*1.0
            confidence/=denom
            if(confidence>thresh):
                new_rules.append([pair[0],pair[1],r,confidence])
    return new_rules

In [146]:
new_rules=prune_rules(rules_dict,thresh=0)
new_rules=sorted(new_rules,reverse=True,key = lambda x: x[3])

In [147]:
print(len(new_rules))
# print(new_rules)
print(new_rules)

3678
[[101, 307, 307, 1.0], [101, 530, 530, 1.0], [144, 818, 144, 1.0], [217, 307, 307, 1.0], [547, 609, 547, 1.0], [807, 609, 807, 1.0], [807, 609, 547, 1.0], [517, 807, 547, 0.9411764705882353], [476, 278, 20, 0.9333333333333333], [66, 147, 676, 0.9259259259259259], [157, 726, 21, 0.9230769230769231], [75, 392, 84, 0.9], [157, 655, 21, 0.8666666666666667], [486, 653, 392, 0.8636363636363636], [3, 1265, 9, 0.8571428571428571], [254, 779, 254, 0.8571428571428571], [517, 84, 547, 0.8571428571428571], [578, 779, 254, 0.8571428571428571], [511, 806, 392, 0.8529411764705882], [486, 812, 392, 0.84375], [42, 473, 84, 0.8421052631578947], [223, 84, 84, 0.8333333333333334], [341, 84, 84, 0.8333333333333334], [704, 84, 84, 0.8333333333333334], [747, 84, 84, 0.8333333333333334], [759, 84, 84, 0.8333333333333334], [1170, 309, 457, 0.8292682926829268], [7, 655, 21, 0.8253968253968254], [13, 1265, 9, 0.8235294117647058], [517, 807, 84, 0.8235294117647058], [39, 392, 84, 0.8181818181818182], [65, 14

In [ ]:
with open(path2,"wb") as writefile:
    pickle.dump(rules_2,writefile)

## Reading and Analysing rules

In [ ]:
def get_inverse_map(mymap):
    inv_map = {}
    for k,v in mymap.items():
        if v in inv_map:
            print("Error, "+str(v)+" is repeated ")
            return
        inv_map[v] = k
    return inv_map

In [ ]:
with open("rules_scores_train.pkl","rb") as f:
    rules_scores = pickle.load(f)
with open("rules_freq_train.pkl","rb") as f:
    rules_freq = pickle.load(f)

In [ ]:
sorted_rules_map = sorted(rules_scores.items(), key=lambda kv: kv[1], reverse=True) 

In [ ]:
rules_inv_map = get_inverse_map(dump['relation_to_id'])

In [ ]:
relation_keyed_data = mine_rules(mapped_train_data)

In [ ]:
def write_rules(filename,freq_thr):
    with open(filename,"w") as f:
        f.write("Score ; Freq_Intersection ; Freq_Union; Freq_Head; Freq_Tail ; Rule Head ; Rule Tail\n")
        for (rprime,r),sc in sorted_rules_map:
            if(rules_freq[(rprime,r)]>=freq_thr):
                union_freq = len(relation_keyed_data[rprime]) + len(relation_keyed_data[r]) - rules_freq[(rprime,r)]
                f.write("%f ; %d ; %d ; %d; %d ; %s ; %s\n" %(
                    sc,rules_freq[(rprime,r)],union_freq,len(relation_keyed_data[rprime]),
                    len(relation_keyed_data[r]),rules_inv_map[rprime],rules_inv_map[r]))

In [ ]:
write_rules("rules_mined.txt",100)